# RNN

In [2]:
# imports 
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
from torchvision.datasets import FashionMNIST

print(f'torch version is : {torch.__version__}')

torch version is : 1.11.0+cpu


In [3]:
# Define device and parameters
epochs = 5
input_size = 28
num_classes = 10
batch_size = 100
hidden_size = 128
num_layers = 2
learning_rate = 0.01
device = 'gpu' if torch.cuda.is_available() else 'cpu'

In [4]:
# Let's get the dataset 
train_dataset = FashionMNIST(
    root='../basics/data',
    transform=ToTensor(),
    train=True,
    download=False
)

test_dataset = FashionMNIST(
    root='../basics/data',
    transform=ToTensor(),
    train=False,
    download=False
)


In [5]:
# Let's define the data loading pipeline
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [7]:
# defien the model 
from turtle import forward


class RNN(nn.Module):
    # Let's deinfe the parameters 
    def __init__(self, input_size=28, hidden_size=128, num_layers=2, num_classes=10, batch_size=100):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.batch_size = batch_size
        
        # Let's define the layers 
        self.lstm = nn.LSTM(
            self.input_size, 
            self.hidden_size,
            self.num_layers, 
            batch_first=True
        )
        # Define the output dense layer
        self.fcn = nn.Linear(in_features=self.hidden_size, out_features=self.num_classes)

    def forward(self, x):
        # Define the intial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate the LSTM
        out, _ = self.lstm(x, (h0, c0))
        out = self.fcn(out[:, -1, :])
        return out

# Let's define the model and initialize it
model = RNN(
    input_size=input_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    num_classes=num_classes,
    batch_size=batch_size
    ).to(device)

In [8]:
# Let's define the loss and optimizer
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Let's define the training loop
from tqdm import tqdm

sequence_length = 28
for epoch in tqdm(range(epochs)):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Let's define the forward pass
        outputs = model(images)
        l = loss(outputs, labels)

        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print(f'epoch is : {epoch}, loss is ; {l.item()}')

 20%|██        | 1/5 [02:11<08:46, 131.65s/it]

epoch is : 0, loss is ; 0.36182305216789246


 40%|████      | 2/5 [04:17<06:24, 128.13s/it]

epoch is : 1, loss is ; 0.2628854811191559


 60%|██████    | 3/5 [06:22<04:13, 126.66s/it]

epoch is : 2, loss is ; 0.2689519226551056


 80%|████████  | 4/5 [08:34<02:09, 129.03s/it]

epoch is : 3, loss is ; 0.2442951798439026


100%|██████████| 5/5 [10:28<00:00, 125.77s/it]

epoch is : 4, loss is ; 0.22812050580978394


In [10]:
# Test the model
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

Test Accuracy of the model on the 10000 test images: 87.91 %
